In [ ]:
#1> load the environment + define the path
%reload_ext autoreload
%autoreload 2

#make data file
print( "Takes <30s. However, it might take >1 mins or so if network is busy...")
import glob
import os
import pandas as pd
from datetime import datetime
import numpy as np
import main_funcs as mfun
import utils_funcs as utils # utils is from Vape - catcher file: 
import matplotlib.pyplot as plt
import pickle



# Get the list of recordings
info = mfun.analysis()
# display the detected session
print( "Total Session: " +  str(info.recordingList .shape[0]))
#info.recordingList.head()

# set matlab API
import matlab.engine
eng = matlab.engine.start_matlab()
print('Matlab engine is set correctly.')

info.recordingList

In [ ]:
#1> OPTIONAL Filter for a particular expRef from the info list
specific_expRef = '2025-05-14_1_MBL015'

# Recreate info object with the specific expRef
info.recordingList = info.recordingList[info.recordingList['sessionName'] == specific_expRef].reset_index(drop=True)
info.recordingList

In [ ]:
#2> cut tiff

import tifffile
import numpy as np
import os

# Define input and output paths
input_file = "//QNAP-AL001.dpag.ox.ac.uk/Data/MBL015/2025-05-14/TwoP/2025-05-14_t-001/2025-05-14_t-001_Cycle00001_Ch2.tif"  # Replace with your input TIFF path
output_file = "//QNAP-AL001.dpag.ox.ac.uk/Data/MBL015/2025-05-14/TwoP/2025-05-14_t-001/2025-05-14_t-001_Cycle00001_Ch2_CUT.tif"   # Replace with your desired output path
frame_to_cut = 40000  # Replace with your desired frame number

# Open the TIFF file
print(f"Opening TIFF file: {input_file}")
with tifffile.TiffFile(input_file) as tif:
    # Get the total number of frames
    total_frames = len(tif.pages)
    print(f"Total frames in the file: {total_frames}")
    
    # Check if the frame number is valid
    if frame_to_cut >= total_frames:
        raise ValueError(f"Frame number {frame_to_cut} is greater than or equal to the total number of frames ({total_frames})")
    
    # Get image dimensions from the first frame
    first_frame = tif.pages[0].asarray()
    height, width = first_frame.shape
    
    # Create output file with the same metadata
    print(f"Creating output file: {output_file}")
    with tifffile.TiffWriter(output_file, bigtiff=True) as tif_out:
        # Process frames in chunks
        chunk_size = 100  # Process 100 frames at a time
        for start_frame in range(0, frame_to_cut, chunk_size):
            end_frame = min(start_frame + chunk_size, frame_to_cut)
            print(f"Processing frames {start_frame} to {end_frame}")
            
            # Read chunk of frames
            frames = np.stack([tif.pages[i].asarray() for i in range(start_frame, end_frame)])
            
            # Save chunk to output file
            tif_out.write(frames)

print(f"New TIFF file saved successfully with {frame_to_cut} frames") 

In [ ]:
#3> cut paq file

import tifffile
import numpy as np
import os
from paq2py import paq_read
from adjust_paq_to_tiff import adjust_paq_to_tiff

# Define the paths of your TIFF and PAQ files
tiff_path = "//QNAP-AL001.dpag.ox.ac.uk/Data/MBL015/2025-05-14/TwoP/2025-05-14_t-001/2025-05-14_t-001_Cycle00001_Ch2.tif"
paq_path = "//QNAP-AL001.dpag.ox.ac.uk/Data/MBL015/2025-05-14/TwoP/2025-05-14_MBL015_1_paq.paq"
output_paq_path = "//QNAP-AL001.dpag.ox.ac.uk/Data/MBL015/2025-05-14/TwoP/2025-05-14_MBL015_1_paq_adjusted.paq"  # optional

new_paqfile = adjust_paq_to_tiff(tiff_path, paq_path, output_paq_path)